<a href="https://colab.research.google.com/github/murilofarias10/Python/blob/main/SmartFillQuiz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gradio PyPDF2 python-pptx

In [15]:
pip install gradio nltk

In [61]:
import gradio as gr
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tag import pos_tag
import random
import os

# Initialize NLTK data
nltk_data_dir = os.path.expanduser('~/nltk_data')
if not os.path.exists(nltk_data_dir):
    os.makedirs(nltk_data_dir)
    for resource in ['punkt', 'averaged_perceptron_tagger', 'stopwords']:
        nltk.download(resource, quiet=True)

def read_txt_file(file):
    try:
        if not file or not file.name.lower().endswith('.txt'):
            return "Error: Please upload a valid TXT file."
        with open(file.name, 'r', encoding='utf-8') as f:
            content = f.read().strip()
        return content if content else "Error: Empty file."
    except Exception as e:
        return f"Error reading file: {str(e)}"

def create_quiz_questions(content):
    """Generate a single fill-in-the-blank question"""
    if not content or isinstance(content, str) and content.startswith("Error"):
        return "Could not generate question. Please check your file."

    # Get valid sentences
    sentences = [s.strip() for s in content.split('\n')
                if len(s.strip()) > 20 and ':' not in s]

    if not sentences:
        return "Not enough content to generate a question."

    # Get all valid words for options
    all_words = set(word.lower() for sentence in sentences
                   for word in sentence.split()
                   if len(word) > 3 and not word.isupper())

    # Select random sentence
    sentence = random.choice(sentences)

    # Get candidate words for blanking
    words = [word for word in sentence.split()
            if len(word) > 3
            and not word.isupper()
            and word.lower() not in ['they', 'their', 'them', 'like', 'with', 'from']]

    if not words:
        return "Could not generate a valid question from the content."

    # Create question
    chosen_word = random.choice(words)
    blank_sentence = sentence.replace(chosen_word, "_____", 1)

    # Get similar words for options
    similar_words = [w for w in all_words
                    if len(w) >= len(chosen_word) - 2
                    and len(w) <= len(chosen_word) + 2
                    and w != chosen_word.lower()]

    if len(similar_words) < 2:
        return "Could not generate enough options."

    # Create options
    wrong_options = random.sample(similar_words, 2)
    options = [chosen_word] + wrong_options
    random.shuffle(options)

    return {
        "question": blank_sentence,
        "options": [f"Option {chr(97 + i)}: {opt}" for i, opt in enumerate(options)],
        "answer": chosen_word,
        "correct_option": f"Option {chr(97 + options.index(chosen_word))}"
    }

def process_file(file):
    """Process file and generate one question"""
    content = read_txt_file(file)
    if isinstance(content, str) and content.startswith("Error"):
        return content

    question = create_quiz_questions(content)
    if isinstance(question, str):
        return question

    output = f"### Fill in the blank:\n{question['question']}\n\n"
    output += "\n".join(question['options']) + "\n"
    output += f"\n<details><summary>👉 Show Answer</summary>\n\n"
    output += f"**{question['answer']} ({question['correct_option']})**\n"
    output += "</details>\n"

    return output

def create_interface():
    with gr.Blocks(css="""
        .question-box {
            font-size: 14px !important;
            line-height: 1.2 !important;
            white-space: pre-wrap !important;
            padding: 10px !important;
            background-color: #2e7d32 !important;
            border-radius: 5px !important;
            color: white !important;
        }
        details {
            padding: 3px;
            background-color: #2e7d32;
            border-radius: 3px;
            margin: 5px 0;
            border: 1px solid #1b5e20;
        }
        summary {
            cursor: pointer;
            color: white;
            font-weight: bold;
            padding: 3px;
            background-color: #2e7d32;
            border-radius: 2px;
        }
        summary:hover { background-color: #1b5e20; }
        details[open] { padding-bottom: 4px; }
        details > *:not(summary) {
            color: white !important;
            background-color: #1b5e20;
            padding: 4px;
            border-radius: 2px;
            margin-top: 3px;
        }
        .question-box ul { margin: 2px 0; }
        .question-box li {
            padding: 1px 0;
            color: white !important;
        }
        .question-box h3 {
            color: white !important;
            margin: 4px 0;
            padding: 4px;
            border-radius: 2px;
        }
        .question-box p {
            color: white !important;
        }
        .main-title {
            font-size: 20px !important;
            font-weight: bold !important;
            color: #2e7d32 !important;
            margin: 0 !important;
            padding: 5px 0 !important;
        }
        .button-row {
            gap: 5px !important;
        }
    """) as demo:
        gr.Markdown("# 📚 Fill in the Blank Quiz", elem_classes=["main-title"])

        with gr.Row():
            file_input = gr.File(
                label="Upload TXT file",
                file_types=[".txt"],
                scale=2
            )
            with gr.Column(scale=1):
                with gr.Row(elem_classes=["button-row"]):
                    submit_btn = gr.Button("New Question", variant="primary", size="sm")
                    clear_btn = gr.Button("Other File", size="sm")

        output = gr.Markdown(elem_classes=["question-box"])

        submit_btn.click(fn=process_file, inputs=[file_input], outputs=[output])
        clear_btn.click(fn=lambda: (None, None), inputs=[], outputs=[file_input, output])

    return demo

if __name__ == "__main__":
    demo = create_interface()
    demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7a2fdf8a05aefc7a09.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
